### 1-2 분류
### 3. 정규화
### 4. 리뷰 쪼개기
### 5. 벡터만들기 (중단)

In [201]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
plt.rc('font', family='AppleGothic')
plt.rc('axes', unicode_minus=False)

import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import MinMaxScaler

import json
import re
from konlpy.tag import Okt
from collections import Counter
import matplotlib
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from sklearn.feature_extraction.text import CountVectorizer


In [202]:
path = './raw_csv/음식_goodpoint.csv'
df = pd.read_csv(path,index_col=0)
df

,Attraction,Good_Point,Selected_People
803,채그로,뷰가 좋아요,146
804,채그로,집중하기 좋아요,62
805,채그로,읽을만한 책이 많아요,61
806,채그로,인테리어가 멋져요,60
807,채그로,커피가 맛있어요,39
...,...,...,...
17442,h450,아늑해요,2
17443,h450,메뉴 구성이 알차요,2
17444,h450,건강한 맛이에요,2
17445,h450,음료가 맛있어요,1


# 1. 애매한 분류를 살펴보자!

In [203]:
 df[df['Good_Point'].str.count('향신료가 강하지 않아요') >= 1]

,Attraction,Good_Point,Selected_People
1726,마복림할머니집,향신료가 강하지 않아요,1
2080,청수정,향신료가 강하지 않아요,2
2176,[백년가게]만석장,향신료가 강하지 않아요,1
2773,우래옥,향신료가 강하지 않아요,3
3329,삼청동수제비,향신료가 강하지 않아요,6
...,...,...,...
16681,밀화,향신료가 강하지 않아요,1
16759,파크 하얏트 서울 코너스톤,향신료가 강하지 않아요,1
16845,챠크라,향신료가 강하지 않아요,2
17242,하단,향신료가 강하지 않아요,2


In [204]:
condition = (df['Good_Point'] == '향신료가 강하지 않아요' ) & (df['Selected_People'] > 3)
df[condition]

,Attraction,Good_Point,Selected_People
3329,삼청동수제비,향신료가 강하지 않아요,6
3668,샤오바오우육면 종로본점,향신료가 강하지 않아요,15
4050,필동면옥,향신료가 강하지 않아요,5
4859,명동칼국수 본점,향신료가 강하지 않아요,7
4908,소문난성수감자탕 별관,향신료가 강하지 않아요,4
5464,황생가칼국수,향신료가 강하지 않아요,4
6403,꽃밥에피다,향신료가 강하지 않아요,7
6491,페르시안궁전,향신료가 강하지 않아요,51
7368,케르반레스토랑,향신료가 강하지 않아요,15
7859,송화산시도삭면,향신료가 강하지 않아요,7


In [205]:
condition = (df['Good_Point'] == '현지 맛에 가까워요' ) & (df['Selected_People'] > 3)
df[condition]

,Attraction,Good_Point,Selected_People
1715,마복림할머니집,현지 맛에 가까워요,4
2766,우래옥,현지 맛에 가까워요,6
3332,삼청동수제비,현지 맛에 가까워요,4
3581,방산 종합시장,현지 맛에 가까워요,6
3663,샤오바오우육면 종로본점,현지 맛에 가까워요,55
4463,울프강스테이크하우스,현지 맛에 가까워요,7
4860,명동칼국수 본점,현지 맛에 가까워요,6
6478,페르시안궁전,현지 맛에 가까워요,806
7355,케르반레스토랑,현지 맛에 가까워요,267
7455,진주집,현지 맛에 가까워요,4


In [206]:
condition = (df['Good_Point'] == '뷰가 좋아요' ) & (df['Selected_People'] > 3)
df[condition]

,Attraction,Good_Point,Selected_People
803,채그로,뷰가 좋아요,146
967,소소한 풍경,뷰가 좋아요,114
1711,마복림할머니집,뷰가 좋아요,22
1757,뮤직컴플렉스 서울,뷰가 좋아요,34
1790,또순이네집,뷰가 좋아요,12
...,...,...,...
17152,알페도,뷰가 좋아요,21
17316,황토추어탕,뷰가 좋아요,6
17355,까사빠보,뷰가 좋아요,95
17379,포포인츠 바이 쉐라톤 조선 서울역 더 이터리,뷰가 좋아요,250


# 2. 분류

In [207]:
categories = {
    "시설 및 환경": ["뷰가 좋아요", "인테리어가 멋져요", "사진이 잘 나와요", "경관이 독특해요",
                    "좌석이 편해요", "매장이 넓어요", "룸이 잘 되어있어요", "야외 공간이 멋져요",
                     "산책로가 잘 되어있어요", "피크닉하기 좋아요", "편의시설이 잘 되어있어요", "공간이 넓어요",
                    "붐비지 않아요", "방문객이 많아요"],

    "음식의 맛과 질": ["커피가 맛있어요", "음료가 맛있어요", "디저트가 맛있어요", "차가 맛있어요",
                      "재료가 신선해요", "건강한 맛이에요", "신선해요", "원산지 표시가 명확해요",
                      "음식이 맛있어요", "양이 많아요", "고기 질이 좋아요", "잡내가 적어요", "빵이 맛있어요", "기본 안주가 좋아요"],

    "청결도와 위생":  ["화장실이 깨끗해요", "매장이 청결해요", "환기가 잘 돼요", "관리가 잘 되어있어요"],

    "서비스" : ["친절해요", "음식이 빨리 나와요", "포장이 깔끔해요", "직접 잘 구워줘요", "주문제작을 잘해줘요", "안전하게 관리해요"],

    "가격과 가치": ["가성비가 좋아요", "코스요리가 알차요", "세트 구성이 알차요", "비싼 만큼 가치있어요", "가격이 합리적이에요"],

    "음식의 다양성": ["특별한 메뉴가 있어요", "메뉴 구성이 알차요", "반찬이 잘 나와요", "술이 다양해요",
                     "코스가 길어요", "먹거리가 풍부해요", "샐러드바가 잘 되어있어요", "부위가 다양해요", "종류가 다양해요"],

    "분위기": ["특별한 날 가기 좋아요", "컨셉이 독특해요", "테마가 재미있어요", "아늑해요",
              "차분한 분위기에요", "오래 머무르기 좋아요", "조용히 쉬기 좋아요", "혼밥하기 좋아요", "혼술하기 좋아요",
              "파티하기 좋아요", "라이브공연이 훌륭해요", "음악이 좋아요", "대화하기 좋아요", "집중하기 좋아요"],

    "접근성": ["대중교통이 편해요", "근처에 갈 곳이 많아요","주차하기 편해요"],

    "특별한 목적": ["단체모임 하기 좋아요", "아이와 가기 좋아요", "부모도 쉬기 좋아요", "읽을만한 책이 많아요",
                   "향신료가 강하지 않아요","현지 맛에 가까워요", "볼거리가 많아요", "반려동물과 가기 좋아요", "선물하기 좋아요",
                   "비주얼이 좋아요", "놀거리가 많아요", "체험 프로그램이 많아요"]

}

In [208]:
for i in df['Good_Point']:
    if i in categories['시설 및 환경']:
        df.loc[df['Good_Point'] == i, 'Category'] = '시설 및 환경'
    elif i in categories['음식의 맛과 질']:
        df.loc[df['Good_Point'] == i, 'Category'] = '음식의 맛과 질'
    elif i in categories['청결도와 위생']:
        df.loc[df['Good_Point'] == i, 'Category'] = '청결도와 위생'
    elif i in categories['서비스']:
        df.loc[df['Good_Point'] == i, 'Category'] = '서비스'
    elif i in categories['가격과 가치']:
        df.loc[df['Good_Point'] == i, 'Category'] = '가격과 가치'
    elif i in categories['음식의 다양성']:
        df.loc[df['Good_Point'] == i, 'Category'] = '음식의 다양성'
    elif i in categories['분위기']:
        df.loc[df['Good_Point'] == i, 'Category'] = '분위기'
    elif i in categories['접근성']:
        df.loc[df['Good_Point'] == i, 'Category'] = '접근성'
    elif i in categories['특별한 목적']:
        df.loc[df['Good_Point'] == i, 'Category'] = '특별한 목적'
    else:
        df.loc[df['Good_Point'] == i, 'Category'] = '분류안됨'

In [209]:
df[df['Category'] == '분류안됨']['Good_Point'].unique()
# 모두 분류됨

array([], dtype=object)

# 3. 정규화 해보자

In [210]:
df

,Attraction,Good_Point,Selected_People,Category
803,채그로,뷰가 좋아요,146,시설 및 환경
804,채그로,집중하기 좋아요,62,분위기
805,채그로,읽을만한 책이 많아요,61,특별한 목적
806,채그로,인테리어가 멋져요,60,시설 및 환경
807,채그로,커피가 맛있어요,39,음식의 맛과 질
...,...,...,...,...
17442,h450,아늑해요,2,분위기
17443,h450,메뉴 구성이 알차요,2,음식의 다양성
17444,h450,건강한 맛이에요,2,음식의 맛과 질
17445,h450,음료가 맛있어요,1,음식의 맛과 질


In [211]:
min_ = df.groupby('Attraction')['Selected_People'].min()
max_ = df.groupby('Attraction')['Selected_People'].max()
sum_ = df.groupby('Attraction')['Selected_People'].sum()

df = pd.merge(df, pd.DataFrame(min_).reset_index(drop=False).rename(columns = {'Selected_People':'min'}))
df = pd.merge(df, pd.DataFrame(max_).reset_index(drop=False).rename(columns = {'Selected_People':'max'}))
df = pd.merge(df, pd.DataFrame(sum_).reset_index(drop=False).rename(columns = {'Selected_People':'sum'}))

df['min_max'] = (df['Selected_People'] - df['min']) / (df['max'] - df['min'])
df['ratio'] = df['Selected_People'] / df['sum']
df.drop(columns=['min','max','sum'],inplace=True)

In [212]:
df[df['min_max'] == 0.000000]


,Attraction,Good_Point,Selected_People,Category,min_max,ratio
16,채그로,차분한 분위기에요,1,분위기,0.0,0.001727
17,채그로,매장이 넓어요,1,시설 및 환경,0.0,0.001727
18,채그로,매장이 청결해요,1,청결도와 위생,0.0,0.001727
19,채그로,건강한 맛이에요,1,음식의 맛과 질,0.0,0.001727
41,소소한 풍경,컨셉이 독특해요,1,분위기,0.0,0.000696
...,...,...,...,...,...,...
10819,포포인츠 바이 쉐라톤 조선 서울역 더 이터리,아이와 가기 좋아요,1,특별한 목적,0.0,0.000313
10820,포포인츠 바이 쉐라톤 조선 서울역 더 이터리,양이 많아요,1,음식의 맛과 질,0.0,0.000313
10837,오양회참치 광화문본점,룸이 잘 되어있어요,1,시설 및 환경,0.0,0.002242
10858,h450,음료가 맛있어요,1,음식의 맛과 질,0.0,0.001181


In [213]:
df[df['ratio'] == 0.000000].groupby('Good_Point')['ratio'].count()
# 어라 생각보다 많아서 큰일이다

Series([], Name: ratio, dtype: int64)

In [214]:
1765/ 10860
# 약 16% 의 데이터..
# drop 해도 괜찮을까...?
# 그룹내 데이터가 적어서 발생하는 문제인것 같다
# 그럼 정규화 말고 비율로 접근해보자

0.1625230202578269

In [215]:
df['ratio'].min()
# 적어도 0 값은 없다

2.2310970304098525e-05

In [216]:
df.drop(columns=['min_max'],inplace=True)

# 4. 쪼개보자

## 4-1 띄어쓰기로 쪼개기

In [217]:
df[['index_0','index_1']] = df['Good_Point'].str.split(n=1,expand=True)
df

,Attraction,Good_Point,Selected_People,Category,ratio,index_0,index_1
0,채그로,뷰가 좋아요,146,시설 및 환경,0.252159,뷰가,좋아요
1,채그로,집중하기 좋아요,62,분위기,0.107081,집중하기,좋아요
2,채그로,읽을만한 책이 많아요,61,특별한 목적,0.105354,읽을만한,책이 많아요
3,채그로,인테리어가 멋져요,60,시설 및 환경,0.103627,인테리어가,멋져요
4,채그로,커피가 맛있어요,39,음식의 맛과 질,0.067358,커피가,맛있어요
...,...,...,...,...,...,...,...
10855,h450,아늑해요,2,분위기,0.002361,아늑해요,None
10856,h450,메뉴 구성이 알차요,2,음식의 다양성,0.002361,메뉴,구성이 알차요
10857,h450,건강한 맛이에요,2,음식의 맛과 질,0.002361,건강한,맛이에요
10858,h450,음료가 맛있어요,1,음식의 맛과 질,0.001181,음료가,맛있어요


## 4-2 명사로 쪼개기

In [218]:
all_values = ' '.join(df['index_0'])

In [219]:
nlp = Okt()
message = nlp.nouns(all_values)
# 형용사 추출
adjectives = []
for word, pos in nlp.pos(all_values):
    # 만약 단어가 명사가 아니라면
    if word not in message:
        # 형용사인 경우에만 리스트에 추가
        if pos == 'Adjective':
            adjectives.append(word)

# 형용사 리스트 출력
print(set(adjectives))

{'조용히', '차분한', '아늑해요', '비싼', '안전하게', '신선해요', '차가', '친절해요', '특별한', '건강한'}


In [220]:
count = Counter(message)

In [221]:
word_count = dict()
tags = []
counting = []
for tag,counts in count.most_common(50):
    if (len(str(tag)) > 1) :
        word_count[tag] = counts
        # print(tag,count)
        tags.append(tag)
        counting.append(counts)

In [222]:
df['word'] = df['index_0'].apply(lambda x: next((tag for tag in tags if tag in x), None))
df

,Attraction,Good_Point,Selected_People,Category,ratio,index_0,index_1,word
0,채그로,뷰가 좋아요,146,시설 및 환경,0.252159,뷰가,좋아요,None
1,채그로,집중하기 좋아요,62,분위기,0.107081,집중하기,좋아요,집중
2,채그로,읽을만한 책이 많아요,61,특별한 목적,0.105354,읽을만한,책이 많아요,None
3,채그로,인테리어가 멋져요,60,시설 및 환경,0.103627,인테리어가,멋져요,인테리어
4,채그로,커피가 맛있어요,39,음식의 맛과 질,0.067358,커피가,맛있어요,커피
...,...,...,...,...,...,...,...,...
10855,h450,아늑해요,2,분위기,0.002361,아늑해요,None,None
10856,h450,메뉴 구성이 알차요,2,음식의 다양성,0.002361,메뉴,구성이 알차요,메뉴
10857,h450,건강한 맛이에요,2,음식의 맛과 질,0.002361,건강한,맛이에요,None
10858,h450,음료가 맛있어요,1,음식의 맛과 질,0.001181,음료가,맛있어요,음료


In [223]:
for i in adjectives:
    df.loc[df['index_0'].str.contains(i), 'word'] = i

In [224]:
df

,Attraction,Good_Point,Selected_People,Category,ratio,index_0,index_1,word
0,채그로,뷰가 좋아요,146,시설 및 환경,0.252159,뷰가,좋아요,None
1,채그로,집중하기 좋아요,62,분위기,0.107081,집중하기,좋아요,집중
2,채그로,읽을만한 책이 많아요,61,특별한 목적,0.105354,읽을만한,책이 많아요,None
3,채그로,인테리어가 멋져요,60,시설 및 환경,0.103627,인테리어가,멋져요,인테리어
4,채그로,커피가 맛있어요,39,음식의 맛과 질,0.067358,커피가,맛있어요,커피
...,...,...,...,...,...,...,...,...
10855,h450,아늑해요,2,분위기,0.002361,아늑해요,None,아늑해요
10856,h450,메뉴 구성이 알차요,2,음식의 다양성,0.002361,메뉴,구성이 알차요,메뉴
10857,h450,건강한 맛이에요,2,음식의 맛과 질,0.002361,건강한,맛이에요,건강한
10858,h450,음료가 맛있어요,1,음식의 맛과 질,0.001181,음료가,맛있어요,음료


In [225]:
# 비어있는 값 확인해보기
df[df['word'].isnull()][['index_0','index_1','word']]

,index_0,index_1,word
0,뷰가,좋아요,None
2,읽을만한,책이 많아요,None
25,뷰가,좋아요,None
30,양이,많아요,None
50,양이,많아요,None
...,...,...,...
10820,양이,많아요,None
10824,양이,많아요,None
10837,룸이,잘 되어있어요,None
10850,양이,많아요,None


In [226]:
df[df['word'].isnull()]['index_0'].unique()

array(['뷰가', '읽을만한', '양이', '잡내가', '술이', '오래', '룸이', '볼거리가', '근처에', '붐비지',
       '방문객이', '피크닉하기', '관리가', '편의시설이', '먹거리가', '빵이', '세트', '비주얼이',
       '놀거리가', '테마가', '부모도', '공간이', '체험', '원산지', '부위가', '라이브공연이'],
      dtype=object)

In [227]:
# Good_Point 검색용
df[df['Good_Point'].str.contains('붐비지')]

,Attraction,Good_Point,Selected_People,Category,ratio,index_0,index_1,word
192,청계산,붐비지 않아요,9,시설 및 환경,0.04918,붐비지,않아요,None


In [228]:
replace_dict = {
    '뷰가': '뷰',
    '읽을만한': '책다양성',
    '친절해요': '친절',
    '양이': '음식양',
    '잡내가': '음식냄새',
    '술이': '술(다양성)',
    '오래': '머무르기좋음',
    '룸이': '룸(시설)',
    '볼거리가': '볼거리',
    '근처에': '근처시설',
    '붐비지': '한산함',
    '방문객이': '방문객',
    '피크닉하기': '피크닉',
    '관리가': '관리',
    '편의시설이': '편의시설',
    '먹거리가': '먹거리',
    '빵이': '빵',
    '세트': '세트(다양성)',
    '비주얼이': '비주얼',
    '놀거리가': '놀거리',
    '테마가': '테마',
    '부모도': '부모',
    '공간이': '공간',
    '체험': '체험(다양성)',
    '원산지': '원산지(다양성)',
    '부위가': '부위',
    '라이브공연이': '라이브공연'
}

# 사전을 기반으로 데이터프레임을 반복하며 대체 작업을 수행합니다.
for key, value in replace_dict.items():
    df.loc[df['index_0'] == key, 'word'] = value

In [229]:
# df.drop(columns='index_1',inplace=True)
df

,Attraction,Good_Point,Selected_People,Category,ratio,index_0,index_1,word
0,채그로,뷰가 좋아요,146,시설 및 환경,0.252159,뷰가,좋아요,뷰
1,채그로,집중하기 좋아요,62,분위기,0.107081,집중하기,좋아요,집중
2,채그로,읽을만한 책이 많아요,61,특별한 목적,0.105354,읽을만한,책이 많아요,책다양성
3,채그로,인테리어가 멋져요,60,시설 및 환경,0.103627,인테리어가,멋져요,인테리어
4,채그로,커피가 맛있어요,39,음식의 맛과 질,0.067358,커피가,맛있어요,커피
...,...,...,...,...,...,...,...,...
10855,h450,아늑해요,2,분위기,0.002361,아늑해요,None,아늑해요
10856,h450,메뉴 구성이 알차요,2,음식의 다양성,0.002361,메뉴,구성이 알차요,메뉴
10857,h450,건강한 맛이에요,2,음식의 맛과 질,0.002361,건강한,맛이에요,건강한
10858,h450,음료가 맛있어요,1,음식의 맛과 질,0.001181,음료가,맛있어요,음료


In [230]:
# 중복된 값이 있는지 확인
duplicates = df[~df.duplicated(subset=['index_0', 'word'], keep=False)]

duplicates

# 그냥 깔끔하게 짤라서 들어간듯?

,Attraction,Good_Point,Selected_People,Category,ratio,index_0,index_1,word
186,청계산,대중교통이 편해요,14,접근성,0.076503,대중교통이,편해요,대중교통
187,청계산,산책로가 잘 되어있어요,14,시설 및 환경,0.076503,산책로가,잘 되어있어요,산책로
188,청계산,경관이 독특해요,12,시설 및 환경,0.065574,경관이,독특해요,경관
190,청계산,볼거리가 많아요,10,특별한 목적,0.054645,볼거리가,많아요,볼거리
191,청계산,근처에 갈 곳이 많아요,10,접근성,0.054645,근처에,갈 곳이 많아요,근처시설
192,청계산,붐비지 않아요,9,시설 및 환경,0.049180,붐비지,않아요,한산함
193,청계산,방문객이 많아요,7,시설 및 환경,0.038251,방문객이,많아요,방문객
194,청계산,피크닉하기 좋아요,5,시설 및 환경,0.027322,피크닉하기,좋아요,피크닉
195,청계산,코스가 길어요,3,음식의 다양성,0.016393,코스가,길어요,코스
196,청계산,관리가 잘 되어있어요,3,청결도와 위생,0.016393,관리가,잘 되어있어요,관리


# 5. NLP

In [231]:
# 주의점! : 단어의 빈도가 우리는 네이버 참여자수의 빈도로 바꿔야한다 !
# TF-IDF 돌리기전 주의!

In [232]:
# 여기서 비율을 넣는게 맞나??
# temp = df[['Attraction', 'ratio', 'word']]
# temp = temp.pivot_table(index='Attraction', columns='word', values='ratio', aggfunc='sum', fill_value=0)
# temp

# 근데 그룹별로 묶이면 안됨!!

In [233]:
temp = df[['ratio', 'word','Attraction']]
# 가로 : 문자 (word)
# 세로 : 장소 (그룹화 되면 안됨)
# 값 : 비율 (이 맞는지 모르겠다. 빈도로 들어가기엔 장소마다 차이가 있고,,, 비율*100으로 갈까?)
temp1 = temp.pivot_table(index='Attraction', columns='word',values='ratio').fillna(0)
temp1

word,가격,가성,건강한,경관,고기,공간,관리,근처시설,기본,놀거리,...,편의시설,포장,피크닉,한산함,향신료,현지,혼밥,혼술,화장실,환기
Attraction,,,,,,,,,,,,,,,,,,,,,
1940금문,0.0,0.014793,0.002959,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.041420,0.0,0.005917,0.0
63뷔페 파빌리온,0.0,0.021497,0.000967,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.001592,0.0,0.026046,0.0
BK볏짚 우대갈비 장안동본점,0.0,0.031933,0.001120,0.0,0.051541,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.001120,0.0,0.010084,0.0
RSG 성수,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0
WANGTHAI,0.0,0.024259,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.001797,0.043127,0.005391,0.0,0.014376,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
황금정원,0.0,0.041972,0.003034,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.000253,0.000000,0.004551,0.0,0.013148,0.0
황생가칼국수,0.0,0.035690,0.007854,0.0,0.000231,0.0,0.0,0.0,0.000116,0.0,...,0.0,0.000116,0.0,0.0,0.000462,0.000116,0.020559,0.0,0.011550,0.0
황토추어탕,0.0,0.159886,0.002855,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.033547,0.0,0.004996,0.0


## 5-1. 단어별 벡터화 (빈도 - > 비율 : 방문자수 )

In [234]:
temp = df[['ratio', 'word','Attraction']]
temp = temp.pivot(columns='word', values='ratio').fillna(0)
temp
# 이걸 이상태로 진행해도 올바른지 의문..

word,가격,가성,건강한,경관,고기,공간,관리,근처시설,기본,놀거리,...,편의시설,포장,피크닉,한산함,향신료,현지,혼밥,혼술,화장실,환기
0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10855,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10856,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10857,0.0,0.0,0.002361,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10858,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 5-2. train/test split

In [235]:
X = temp.values
X

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.00236128, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [236]:
y = df['Category']
y
# 근데 이게 y 값이면 그룹별로 묶으면 안됨!
# y 값을 어떻게 해야되냐 도대체

0         시설 및 환경
1             분위기
2          특별한 목적
3         시설 및 환경
4        음식의 맛과 질
           ...   
10855         분위기
10856     음식의 다양성
10857    음식의 맛과 질
10858    음식의 맛과 질
10859         서비스
Name: Category, Length: 10860, dtype: object

In [237]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [238]:
x_train.shape, x_test.shape,y_train.shape,y_test.shape

((8688, 77), (2172, 77), (8688,), (2172,))

## 5-3. 학습

In [239]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()
model

DecisionTreeClassifier()

In [240]:
# fit(기출문제, 정답)
model.fit(x_train,y_train)

DecisionTreeClassifier()

## 5-4. 예측

In [241]:
# predict(실전문제)
y_pred = model.predict(x_test)
y_pred

array(['음식의 맛과 질', '음식의 맛과 질', '가격과 가치', ..., '시설 및 환경', '시설 및 환경', '분위기'],
      dtype=object)

## 5-5. 평가

In [242]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)
# 93% 의 정확도를 가지고 분류됨

0.932780847145488

In [243]:
pd.crosstab(y_test,y_pred)
# 얼마나 올바르게 예측되었는지
# confustion matrix
# 대부분 잘 예측됨을 파악할 수 있다
# 분위기 - 음식의 다양성
# 시설및 환경 - 청결도 위생 의 경우 어느정도 분류를 잘 못한 부분이 보이나
# 비율로 보았을때 크진 않아보이긴 함
# pca 로 차원축소를 할때 합쳐질 가능성도 있을것 같다

col_0,가격과 가치,분위기,서비스,시설 및 환경,음식의 다양성,음식의 맛과 질,접근성,청결도와 위생,특별한 목적
Category,,,,,,,,,
가격과 가치,171,0,0,0,0,1,0,0,0
분위기,0,340,0,0,34,1,0,0,0
서비스,0,0,169,0,0,8,0,0,0
시설 및 환경,0,0,0,323,0,0,0,47,0
음식의 다양성,0,26,0,0,182,0,0,0,0
음식의 맛과 질,0,0,3,0,0,461,0,0,0
접근성,0,0,0,0,0,0,86,0,0
청결도와 위생,0,0,0,26,0,0,0,159,0
특별한 목적,0,0,0,0,0,0,0,0,135


### 그런데 여기서 피처를 줄이지 않고 진행하였는데, pca해서 피처를 줄이고 진행해도 괜찮을것 같습니다

### 각 단어가 해당 카테고리에 잘 분류되었는지를 확인하는데 방문자 수를 가지고 벡터화를 해서 수치화 한다음에 저희가 분류한 카테고리랑 피팅하는게.... 맞나요...?? 이 자꾸 헷갈리네... 그리고 y 를 그룹화를 못해서 전체 데이터 대상으로 했는데...(어차피 장소별로 리뷰카테고리가 중요해서) 장소를 그룹별로 안묶어도 괜찮나요...??

# 6. 트리 알고리즘

In [244]:
from sklearn.tree import export_text
print(export_text(model))

|--- feature_1 <= 0.00
|   |--- feature_60 <= 0.01
|   |   |--- feature_53 <= 0.00
|   |   |   |--- feature_10 <= 0.00
|   |   |   |   |--- feature_75 <= 0.00
|   |   |   |   |   |--- feature_16 <= 0.00
|   |   |   |   |   |   |--- feature_47 <= 0.00
|   |   |   |   |   |   |   |--- feature_26 <= 0.00
|   |   |   |   |   |   |   |   |--- feature_27 <= 0.00
|   |   |   |   |   |   |   |   |   |--- feature_19 <= 0.00
|   |   |   |   |   |   |   |   |   |   |--- feature_65 <= 0.00
|   |   |   |   |   |   |   |   |   |   |   |--- truncated branch of depth 52
|   |   |   |   |   |   |   |   |   |   |--- feature_65 >  0.00
|   |   |   |   |   |   |   |   |   |   |   |--- truncated branch of depth 31
|   |   |   |   |   |   |   |   |   |--- feature_19 >  0.00
|   |   |   |   |   |   |   |   |   |   |--- class: 음식의 다양성
|   |   |   |   |   |   |   |   |--- feature_27 >  0.00
|   |   |   |   |   |   |   |   |   |--- class: 가격과 가치
|   |   |   |   |   |   |   |--- feature_26 >  0.00
|   |   |   | 

# ~ 여기서 이어서 진행해보자~

In [245]:
df

,Attraction,Good_Point,Selected_People,Category,ratio,index_0,index_1,word
0,채그로,뷰가 좋아요,146,시설 및 환경,0.252159,뷰가,좋아요,뷰
1,채그로,집중하기 좋아요,62,분위기,0.107081,집중하기,좋아요,집중
2,채그로,읽을만한 책이 많아요,61,특별한 목적,0.105354,읽을만한,책이 많아요,책다양성
3,채그로,인테리어가 멋져요,60,시설 및 환경,0.103627,인테리어가,멋져요,인테리어
4,채그로,커피가 맛있어요,39,음식의 맛과 질,0.067358,커피가,맛있어요,커피
...,...,...,...,...,...,...,...,...
10855,h450,아늑해요,2,분위기,0.002361,아늑해요,None,아늑해요
10856,h450,메뉴 구성이 알차요,2,음식의 다양성,0.002361,메뉴,구성이 알차요,메뉴
10857,h450,건강한 맛이에요,2,음식의 맛과 질,0.002361,건강한,맛이에요,건강한
10858,h450,음료가 맛있어요,1,음식의 맛과 질,0.001181,음료가,맛있어요,음료


In [246]:
model.feature_importances_

array([0.00026393, 0.05018059, 0.        , 0.00026794, 0.        ,
       0.00026757, 0.00026411, 0.00026447, 0.        , 0.00026628,
       0.04860084, 0.00026411, 0.02614921, 0.        , 0.        ,
       0.01668935, 0.09482732, 0.01541624, 0.00026483, 0.03167196,
       0.0069933 , 0.02717589, 0.00026645, 0.00026665, 0.00026517,
       0.00026446, 0.04403392, 0.0336793 , 0.00026554, 0.        ,
       0.01401869, 0.0002672 , 0.00131157, 0.00233839, 0.        ,
       0.0077747 , 0.        , 0.0261068 , 0.01400989, 0.00026519,
       0.01405015, 0.        , 0.        , 0.0492027 , 0.        ,
       0.        , 0.01010942, 0.04319048, 0.        , 0.00026482,
       0.000265  , 0.00776022, 0.0165501 , 0.04967149, 0.0139494 ,
       0.01167912, 0.        , 0.02568382, 0.00052531, 0.00026591,
       0.05001394, 0.        , 0.0221136 , 0.01418163, 0.00026537,
       0.08281241, 0.00026573, 0.00026682, 0.00764934, 0.00026571,
       0.00026608, 0.01248601, 0.0116811 , 0.03034707, 0.00693

# 7. 벡터화

In [247]:
vectorizer = CountVectorizer()

In [248]:
lst = list()
for i in df['word'] :
    lst.append(i)

In [249]:
dtm = vectorizer.fit_transform(lst)
tf = pd.DataFrame(dtm.toarray(), columns=vectorizer.get_feature_names_out())

In [250]:
tf

,가격,가성,건강한,경관,고기,공간,관리,근처시설,기본,놀거리,...,편의시설,포장,피크닉,한산함,향신료,현지,혼밥,혼술,화장실,환기
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10855,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10856,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10857,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10858,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [251]:
 df.at[index, 'Selected_People']

1

In [252]:
def multiply_selected_people(tf_row, multiplier):
    return tf_row * multiplier

# 곱셈 적용
for index, tf_row in tf.iterrows():
    selected_people = df.at[index, 'Selected_People']
    tf.loc[index] = multiply_selected_people(tf_row, selected_people)

print(tf)

       가격  가성  건강한  경관  고기  공간  관리  근처시설  기본  놀거리  ...  편의시설  포장  피크닉  한산함  \
0       0   0    0   0   0   0   0     0   0    0  ...     0   0    0    0   
1       0   0    0   0   0   0   0     0   0    0  ...     0   0    0    0   
2       0   0    0   0   0   0   0     0   0    0  ...     0   0    0    0   
3       0   0    0   0   0   0   0     0   0    0  ...     0   0    0    0   
4       0   0    0   0   0   0   0     0   0    0  ...     0   0    0    0   
...    ..  ..  ...  ..  ..  ..  ..   ...  ..  ...  ...   ...  ..  ...  ...   
10855   0   0    0   0   0   0   0     0   0    0  ...     0   0    0    0   
10856   0   0    0   0   0   0   0     0   0    0  ...     0   0    0    0   
10857   0   0    2   0   0   0   0     0   0    0  ...     0   0    0    0   
10858   0   0    0   0   0   0   0     0   0    0  ...     0   0    0    0   
10859   0   0    0   0   0   0   0     0   0    0  ...     0   0    0    0   

       향신료  현지  혼밥  혼술  화장실  환기  
0        0   0   0   0    0  

In [253]:
tf 
# 방문자수가 곱해진 벡터

,가격,가성,건강한,경관,고기,공간,관리,근처시설,기본,놀거리,...,편의시설,포장,피크닉,한산함,향신료,현지,혼밥,혼술,화장실,환기
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10855,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10856,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10857,0,0,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10858,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [254]:
# list(tf.iloc[1,:])
# 확인 완료

In [255]:
tf_df = tf.astype(bool).sum(axis = 0)
tf_df

가격       2
가성     456
건강한    336
경관       1
고기     167
      ... 
현지      74
혼밥     379
혼술      37
화장실    433
환기      35
Length: 75, dtype: int64

# 8. TF-IDF

In [256]:
# idf

# 문서 개수
D = len(tf)

# Inverse Document Frequency
idf = np.log((D) / (tf_df+1))

In [257]:
# TF-IDF
tfidf = tf * idf                      
tfidf = tfidf / np.linalg.norm(tfidf, axis = 1, keepdims = True)

In [258]:
tf_idf = tfidf.fillna(0)
tf_idf

,가격,가성,건강한,경관,고기,공간,관리,근처시설,기본,놀거리,...,편의시설,포장,피크닉,한산함,향신료,현지,혼밥,혼술,화장실,환기
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10855,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10856,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10857,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10858,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# 9. 유사도 거리계산

## 9-1. 코사인 유사도

In [259]:
tf_idf.index = df['Attraction']
tf_idf

,가격,가성,건강한,경관,고기,공간,관리,근처시설,기본,놀거리,...,편의시설,포장,피크닉,한산함,향신료,현지,혼밥,혼술,화장실,환기
Attraction,,,,,,,,,,,,,,,,,,,,,
채그로,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
채그로,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
채그로,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
채그로,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
채그로,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
h450,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
h450,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
h450,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


그런데 한 행마다 값이 1개씩 밖에 없을텐데... 이게 맞나??  
기존의 문장 -> 벡터화는 하나의 문장을 쪼개서 여러가지 단어가 카운트 되는데 ...   
그러니까 우리가 가진 데이터안에는 중복으로 카운트 되는 값이 없어서 ......  
이걸로 접근하는게 맞나...? 싶은 생각이   
장소기준으로 묶어볼까?

In [261]:
tf_idf.columns[0:]

Index(['가격', '가성', '건강한', '경관', '고기', '공간', '관리', '근처시설', '기본', '놀거리', '다양성',
       '단체', '대중교통', '대화', '디저트', '라이브공연', '매장', '머무르기좋음', '먹거리', '메뉴', '반려동물',
       '반찬', '방문객', '볼거리', '부모', '부위', '비싼', '비주얼', '사진', '산책로', '샐러드', '선물',
       '세트', '시설', '신선해요', '아늑해요', '아이', '안전하게', '야외', '원산지', '음료', '음식',
       '음식냄새', '음식양', '음악', '인테리어', '재료', '제작', '조용히', '종류', '좌석', '주차', '직접',
       '집중', '차가', '차분한', '책다양성', '체험', '친절', '커피', '컨셉', '코스', '테마', '특별한',
       '파티', '편의시설', '포장', '피크닉', '한산함', '향신료', '현지', '혼밥', '혼술', '화장실', '환기'],
      dtype='object')

In [265]:
tf_idf_temp = tf_idf.groupby('Attraction')[['가격', '가성', '건강한', '경관', '고기', '공간', '관리', '근처시설', '기본', '놀거리', '다양성',
       '단체', '대중교통', '대화', '디저트', '라이브공연', '매장', '머무르기좋음', '먹거리', '메뉴', '반려동물',
       '반찬', '방문객', '볼거리', '부모', '부위', '비싼', '비주얼', '사진', '산책로', '샐러드', '선물',
       '세트', '시설', '신선해요', '아늑해요', '아이', '안전하게', '야외', '원산지', '음료', '음식',
       '음식냄새', '음식양', '음악', '인테리어', '재료', '제작', '조용히', '종류', '좌석', '주차', '직접',
       '집중', '차가', '차분한', '책다양성', '체험', '친절', '커피', '컨셉', '코스', '테마', '특별한',
       '파티', '편의시설', '포장', '피크닉', '한산함', '향신료', '현지', '혼밥', '혼술', '화장실', '환기']].sum()

In [267]:
tf_idf_temp[tf_idf_temp['Attraction'] == '1940금문']

KeyError: 'Attraction'